In [1]:
import nltk, re, pprint
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("D:\ML\Datasets\labeled_lyrics_cleaned.csv")

In [3]:
data.head()

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371


In [4]:
data.drop(labels="Unnamed: 0", axis=1, inplace=True)
data.rename(columns={"seq": "lyrics", "label": "valency"}, inplace=True)

#data = data.drop_duplicates(subset=['lyrics', 'song'])
data.sort_values(by=['song', 'valency'], ascending=False, inplace=True) # to keep highest value valency 
data = data.drop_duplicates(subset='lyrics')
pd.set_option('display.float_format', lambda x: '%.2f' % x) # round everything to 2 decimal places
data = data.reset_index() 
data.head()

,index,artist,lyrics,song,valency
0,54911,Simon & Milo,"Hello, this is Stacy, the computer\nGood morni...",www.nevergetoveryou,0.68
1,82479,Hippo Campus,See how the western kids\r\nHave silicon insid...,western kids,0.52
2,82478,Hippo Campus,"Wisconsin pines, collaborating with the day gl...",way it goes,0.52
3,82477,Hippo Campus,"I see meaning where you don't, where you don't...",vines,0.66
4,82476,Hippo Campus,My thoughts are a battlefield of sub-surreal a...,vacation,0.55


In [5]:
from nltk import tokenize
from nltk.corpus import brown, stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import webtext

lyrics = [x.replace("\n", " ").strip().replace("\r", "") for x in data['lyrics']]

In [6]:
#lyrics

In [71]:
import string # removing punctuation

pattern = r'\[.*?\]' # remove square brackets and contents

song = data['lyrics'][3334].replace("\n", " ").replace("\r", "").replace("\t", "").replace("  ", " ").strip()
#song = song.translate(str.maketrans('', '', string.punctuation))
song = re.sub(pattern, '', song)
song

'If you Come close And hold me tight You feel The heart that beats For you  And if You dear Could read my mind Oh you Would know my love Is true   Words can`t say how much I love you Words can`t say how much I care I need you `n I need your love Like I need to breathe the air  Faith And trust Give both a try So you Will see that is The key  I swear True love Will never die So please Believe in you And me '

In [8]:
from nnsplit import NNSplit

splitter = NNSplit.load("en")

In [72]:
splits = splitter.split([song.lower()])[0]
song_sents = [str(x).strip() for x in splits]
song_sents = sorted(set(song_sents))

song_sents

['and if you dear could read my mind oh you would know my love is true',
 'faith and trust give both a try so you will see that is the key',
 'i need to breathe the air',
 'i need you `n',
 'i need your love like',
 'i swear true love will never die so please believe in you and me',
 'if you come close and hold me tight you feel the heart that beats for you',
 'words can`t say how much i love you words can`t say how much i care']

In [73]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()    
tokens = [tokenize.word_tokenize(sent) for sent in song_sents] # should be no punct

In [74]:
from nltk import pos_tag, word_tokenize
pos_tag(word_tokenize(song_sents[0])) 

[('and', 'CC'),
 ('if', 'IN'),
 ('you', 'PRP'),
 ('dear', 'VBP'),
 ('could', 'MD'),
 ('read', 'VB'),
 ('my', 'PRP$'),
 ('mind', 'NN'),
 ('oh', 'IN'),
 ('you', 'PRP'),
 ('would', 'MD'),
 ('know', 'VB'),
 ('my', 'PRP$'),
 ('love', 'NN'),
 ('is', 'VBZ'),
 ('true', 'JJ')]

In [75]:
song = song.translate(str.maketrans('', '', string.punctuation))
#pos_tag(word_tokenize(song))

In [76]:
from textblob import TextBlob
from textblob.parsers import PatternParser

b = TextBlob(song_sents[0])
print(b.parse())

and/CC/O/O if/IN/B-PP/B-PNP you/PRP/B-NP/I-PNP dear/JJ/B-ADJP/O could/MD/B-VP/O read/VB/I-VP/O my/PRP$/B-NP/O mind/NN/I-NP/O oh/UH/O/O you/PRP/B-NP/O would/MD/B-VP/O know/VB/I-VP/O my/PRP$/B-NP/O love/NN/I-NP/O is/VBZ/B-VP/O true/JJ/B-ADJP/O


In [77]:
import spacy
#import textacy

nlp = spacy.load('en_core_web_sm')

In [78]:
doc = nlp(song_sents[0])

for x in doc:
    print(x, spacy.explain(x.tag_))

and conjunction, coordinating
if conjunction, subordinating or preposition
you pronoun, personal
dear adjective (English), other noun-modifier (Chinese)
could verb, modal auxiliary
read verb, base form
my pronoun, possessive
mind noun, singular or mass
oh interjection
you pronoun, personal
would verb, modal auxiliary
know verb, base form
my pronoun, possessive
love noun, singular or mass
is verb, 3rd person singular present
true adjective (English), other noun-modifier (Chinese)
